In [1]:
using Revise
using Pkg

ENV["PYTHON"] = Sys.which("python")
ENV["PYCALL_JL_RUNTIME_PYTHON"] = Sys.which("python")
Pkg.build("PyCall")
using FileIO
using JLD2
include("../src/DistributionallyRobust.jl")
using .DistributionallyRobust

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/8c86e48c0db1564a1d49548d3515ced5d604c408/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/62f417f6ad727987c755549e9cd88c46578da562/build.log`


┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /home/kang/DRC_Project/DRC_Trajectron/Manifest.toml:0


┌ Info: Number of Julia Thread(s): 1
└ @ Main.DistributionallyRobust /home/kang/DRC_Project/DRC_Trajectron/src/DistributionallyRobust.jl:39
┌ Info: CUDA Device: NVIDIA GeForce RTX 3070 Laptop GPU
└ @ Main.DistributionallyRobust /home/kang/DRC_Project/DRC_Trajectron/src/DistributionallyRobust.jl:40


┌ Info: Python executable used by PyCall: /home/kang/anaconda3/envs/trajectron++/bin/python
└ @ Main.DistributionallyRobust /home/kang/DRC_Project/DRC_Trajectron/src/DistributionallyRobust.jl:41


In [8]:
Pkg.add("CSV")
Pkg.add("DataFrames")

using CSV
using DataFrames

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


┌ Warning: The active manifest file at `/home/kang/DRC_Project/DRC_Trajectron/Manifest.toml` has an old format that is being maintained.
│ To update to the new format, which is supported by Julia versions ≥ 1.6.2, run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
│ To then record the julia version re-resolve with `Pkg.resolve()` and if there are resolve conflicts consider `Pkg.update()`.
└ @ Pkg.Types /cache/build/default-amdci4-2/julialang/julia-release-1-dot-8/usr/share/julia/stdlib/v1.8/Pkg/src/manifest.jl:287
  No Changes

 to `~/DRC_Project/DRC_Trajectron/Project.toml`


  No Changes to `~/DRC_Project/DRC_Trajectron/Manifest.toml`


   Resolving package versions...


  No Changes to `~/DRC_Project/DRC_Trajectron/Project.toml`
  No Changes to `~/DRC_Project/DRC_Trajectron/Manifest.toml`


# Experiments

In [15]:
include("$(@__DIR__)/../scripts/default_params/params_drc_sensitivity_gaussian.jl");

epsilon = 0.1;

include("$(@__DIR__)/../scripts/parameter_setup_drc.jl");

collision = [];
position_cost = [];
control_cost = [];
minimum_dist_list = [];

In [16]:
for experiment_iter in 1:100
    print("Experiment $experiment_iter: ")

    scene_loader, controller, w_init, measurement_schedule, target_trajectory, target_speed = 
    controller_setup(scene_param,
                     predictor_param,
                     prediction_device=prediction_device,
                     cost_param=cost_param,
                     cnt_param=cnt_param,
                     dtc=dtc,
                     dto=dto,
                     ado_pos_init_dict=ado_pos_init_dict,
                     ado_vel_dict=ado_vel_dict,
                     ego_pos_init_vec=ego_pos_init_vec,
                     ego_pos_goal_vec=ego_pos_goal_vec,
                     target_speed=target_speed,
                     sim_horizon=sim_horizon,
                     verbose=false);

    result, ~, ~ = evaluate(scene_loader, controller, w_init, ego_pos_goal_vec,
                    target_speed, measurement_schedule, target_trajectory,
                    pos_error_replan);

    minimum_dist = minimum([minimum(vcat([norm(get_position(w.e_state) - ap) for ap in values(w.ap_dict)], Inf))
                                for w in result.w_history])
    
    
    print("Total Collision: $(result.total_col), ")
    print("Total Position Cost: $(result.total_pos_cost), ")
    print("Total Control Cost: $(result.total_cnt_cost), ")
    print("Minimum Distance: $minimum_dist, ")

    push!(collision, result.total_col)
    push!(position_cost, result.total_pos_cost)
    push!(control_cost, result.total_cnt_cost)
    push!(minimum_dist_list, minimum_dist)

    # make_gif(result, dtplot=0.1, fps=5, xlim=(-6., 6.), ylim=(-6., 6.), figsize=(600, 400), 
    #             legendfontsize=7, legend=:topright, markersize=5., filename="rssac_data_trajectron_$(experiment_iter).gif",
    #             show_nominal_trajectory=false)


    if experiment_iter % 50 == 0
        CSV.write("trajectron_drc_$(experiment_iter).csv", DataFrame(collision=collision, position_cost=position_cost, control_cost=control_cost, minimum_dist=minimum_dist_list))
    end
end

Experiment 1: 

Total Collision: 0, Total Position Cost: 134.52169416139668, Total Control Cost: 1.1389516112413731, Minimum Distance: 1.8084420137306547, Experiment 2: 

Total Collision: 0, Total Position Cost: 185.56217503804012, Total Control Cost: 1.193123677370228, Minimum Distance: 0.730676779082661, Experiment 3: 

Total Collision: 0, Total Position Cost: 157.05934744544487, Total Control Cost: 1.2206900362973037, Minimum Distance: 0.7825070507532831, Experiment 4: 

Total Collision: 0, Total Position Cost: 166.0764172055001, Total Control Cost: 1.25252535767615, Minimum Distance: 0.8542754400716278, Experiment 5: 

...

Total Collision: 0, Total Position Cost: 168.7598248220452, Total Control Cost: 1.1092603231830367, Minimum Distance: 0.9562375932258094, Experiment 97: 

Total Collision: 0, Total Position Cost: 169.11508367813713, Total Control Cost: 1.1578416944128336, Minimum Distance: 0.7722422846793146, Experiment 98: 

Total Collision: 0, Total Position Cost: 173.6246861492905, Total Control Cost: 1.2666446648500547, Minimum Distance: 0.815598104089557, Experiment 99: 

Total Collision: 0, Total Position Cost: 166.61411038113917, Total Control Cost: 1.2902130795679314, Minimum Distance: 0.7766240730966748, Experiment 100: 

Total Collision: 0, Total Position Cost: 155.75324853232897, Total Control Cost: 1.2613381459580988, Minimum Distance: 0.5560322247551338, 

In [17]:
make_gif(result, dtplot=0.1, fps=50, xlim=(-6., 6.), ylim=(-6., 6.), figsize=(600, 400), 
         legendfontsize=7, legend=:topright, markersize=5., filename="1_synthetic_gaussian.gif",
         show_nominal_trajectory=false)